# Part_1_5_Words_Corpora_Text_Normalization
In Natural Language Processing (NLP), key techniques such as tokenization, stemming, and sentence segmentation are fundamental for transforming raw text into a structured format that can be effectively analyzed by language models. Byte-Pair Encoding (BPE) tokenization helps handle rare words and out-of-vocabulary terms by breaking text into subword units. Stemming, using algorithms like the Porter Stemmer, reduces words to their root forms, ensuring consistency across the text for better analysis. Sentence segmentation is crucial for dividing text into meaningful sentences, allowing for more accurate processing in downstream tasks. These techniques play a vital role in preparing text for various NLP tasks, ensuring that language data is in a normalized and analyzable state.

### **Objectives:**
By the end of this notebook, Parham will have a thorough understanding of tokenization and its importance in NLP, specifically learning how to implement **Byte-Pair Encoding (BPE)** to handle words and subwords. He will explore the **Porter Stemmer**, gaining insight into how it reduces words to their base forms and why this is essential for text normalization. Additionally, Parham will learn to apply **Sentence Segmentation** to split text into meaningful sentences for deeper analysis. Through hands-on coding exercises, he will gain practical experience using these techniques, utilizing Python libraries like `NLTK` and `SpaCy` to prepare text for NLP tasks.

**Table of Contents:** 
1. Import Libraries
2. Tokenization Techniques
3. Text Normalization: Porter Stemmer
4. Sentence Segmentation
5. Closing Thoughts

## 1. Import Libraries

In [7]:
import re
import os
import sys
from loguru import logger
import nltk
import numpy as np
from collections import Counter, defaultdict

## 2. Tokenization Techniques
Byte-pair encoding was first introduced in 1994 as a simple data compression technique by iteratively replacing the most frequent pair of bytes in a sequence with a single, unused byte.
Imagine you’re reading a really big book, but some of the words are really long or tricky, and you might not know all of them. To make things easier, Parham can break the long words into smaller, simpler parts or pieces. This way, Parham can still understand the book without needing to know every single big word.

Byte-Pair Encoding (BPE) is a popular technique used in natural language processing for tokenizing text into subword units. The idea is to break down words into smaller, more frequent pieces, allowing models to efficiently handle rare or unknown words. BPE is especially useful in scenarios where the vocabulary is limited but the text contains a large variety of words, including compound or out-of-vocabulary words.

The algorithm works by:
- Starting with a sequence of individual characters.
- Finding the most frequent pair of consecutive characters (or subwords).
- Merging this pair into a single token.
- Repeating the process until a predefined number of merges or a desired vocabulary size is reached.

This process allows models to represent both common words and subwords effectively, making it easier to process any text.

In the cell below Parham will an example of BEP tokenization.

In [26]:
# Example Corpus (small set of words)
corpus = ["low", "lowest", "newer", "wider"]

# Create initial vocabulary from characters in the corpus
def get_vocab(corpus):
    vocab = defaultdict(int)
    for word in corpus:
        # Add a space between each character, so we can merge pairs
        word = " ".join(list(word)) + " </w>"
        vocab[word] += 1
    return vocab

# Get the most frequent pair of symbols in the vocabulary
def get_stats(vocab):
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += freq
    return pairs

# Merge the most frequent pair in the vocabulary
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = ' '.join(pair)
    replacement = ''.join(pair)
    for word in v_in:
        # Replace the pair with the merged version
        new_word = word.replace(bigram, replacement)
        v_out[new_word] = v_in[word]
    return v_out

# Perform Byte-Pair Encoding
def byte_pair_encoding(corpus, num_merges):
    vocab = get_vocab(corpus)
    print("Initial Vocabulary:", vocab)
    
    for i in range(num_merges):
        pairs = get_stats(vocab)
        if not pairs:
            break
        best_pair = max(pairs, key=pairs.get)
        vocab = merge_vocab(best_pair, vocab)
        print(f"\nAfter merge {i + 1}:")
        print(f"Best Pair: {best_pair}")
        print(f"Updated Vocabulary: {vocab}")
        
    return vocab

# Run BPE with 10 merges on the example corpus
byte_pair_encoding(corpus, num_merges=10)


Initial Vocabulary: defaultdict(<class 'int'>, {'l o w </w>': 1, 'l o w e s t </w>': 1, 'n e w e r </w>': 1, 'w i d e r </w>': 1})

After merge 1:
Best Pair: ('l', 'o')
Updated Vocabulary: {'lo w </w>': 1, 'lo w e s t </w>': 1, 'n e w e r </w>': 1, 'w i d e r </w>': 1}

After merge 2:
Best Pair: ('lo', 'w')
Updated Vocabulary: {'low </w>': 1, 'low e s t </w>': 1, 'n e w e r </w>': 1, 'w i d e r </w>': 1}

After merge 3:
Best Pair: ('e', 'r')
Updated Vocabulary: {'low </w>': 1, 'low e s t </w>': 1, 'n e w er </w>': 1, 'w i d er </w>': 1}

After merge 4:
Best Pair: ('er', '</w>')
Updated Vocabulary: {'low </w>': 1, 'low e s t </w>': 1, 'n e w er</w>': 1, 'w i d er</w>': 1}

After merge 5:
Best Pair: ('low', '</w>')
Updated Vocabulary: {'low</w>': 1, 'low e s t </w>': 1, 'n e w er</w>': 1, 'w i d er</w>': 1}

After merge 6:
Best Pair: ('low', 'e')
Updated Vocabulary: {'low</w>': 1, 'lowe s t </w>': 1, 'n e w er</w>': 1, 'w i d er</w>': 1}

After merge 7:
Best Pair: ('lowe', 's')
Updated V

{'low</w>': 1, 'lowest</w>': 1, 'ne w er</w>': 1, 'w i d er</w>': 1}

## 3. Text Normalization: Porter Stemmer

## 4. Sentence Segmentation

## 5. Closing Thoughts